In [1]:
#%pip install --upgrade mosek

In [ ]:
from multi_stock_downloader import MultiStockDownloader

import sys, os, re, glob
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# Options
np.set_printoptions(precision=5, linewidth=120, suppress=True)
pd.set_option("display.max_rows", None)
plt.rcParams["figure.figsize"] = [12, 8]

# Diagnostic
print(f"Python: {sys.version}")
print(f"matplotlib: {matplotlib.__version__}, pandas: {pd.__version__}, numpy: {np.__version__}, mosek: {Model.getVersion()}")


Python: 3.12.12 | packaged by conda-forge | (main, Jan 27 2026, 00:01:15) [Clang 19.1.7 ]
matplotlib: 3.10.8, pandas: 2.3.3, numpy: 2.4.2, mosek: 11.1.6


In [3]:
dl = MultiStockDownloader(
    tickers=['AAPL', 'AMZN', 'NVDA', 'MSFT', 'GOOGL'],
    start_date='2023-01-01'
)
df = dl.download()
print(df.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Successfully downloaded 788 records
Successfully downloaded 788 records


[*********************100%***********************]  1 of 1 completed


Successfully downloaded 788 records


[*********************100%***********************]  1 of 1 completed


Successfully downloaded 788 records


[*********************100%***********************]  1 of 1 completed

Successfully downloaded 788 records
                  AAPL       AMZN       NVDA        MSFT      GOOGL
Date                                                               
2023-01-03  123.096031  85.820000  14.300682  233.452805  88.451691
2023-01-04  124.365662  85.139999  14.734251  223.240814  87.419479
2023-01-05  123.046806  83.120003  14.250734  216.624435  85.553581
2023-01-06  127.574203  86.080002  14.844140  219.177460  86.685036
2023-01-09  128.095840  87.360001  15.612371  221.311478  87.359924


In [4]:
df.tail()

,AAPL,AMZN,NVDA,MSFT,GOOGL
Date,,,,,
2026-02-18,264.350006,204.789993,187.979996,398.690002,303.329987
2026-02-19,260.579987,204.860001,187.899994,398.459991,302.850006
2026-02-20,264.579987,210.110001,189.820007,397.230011,314.980011
2026-02-23,266.179993,205.270004,191.550003,384.470001,311.489990
2026-02-24,272.140015,208.559998,192.850006,389.000000,310.899994


In [5]:
df_weekly_prices = df.resample('W').last()
df_weekly_prices.head()

,AAPL,AMZN,NVDA,MSFT,GOOGL
Date,,,,,
2023-01-08,127.574203,86.080002,14.844140,219.177460,86.685036
2023-01-15,132.633072,98.120003,16.882099,233.111740,91.429192
2023-01-22,135.694000,97.250000,17.821163,234.076431,97.284935
2023-01-29,143.626770,102.239998,20.344635,241.813370,98.624832
2023-02-05,152.061539,103.389999,21.078899,251.742783,103.994247


In [6]:
df_weekly_log_returns = \
    np.log(df_weekly_prices) - np.log(df_weekly_prices.shift(1))
df_weekly_log_returns = df_weekly_log_returns.dropna(how='all')
df_weekly_log_returns = df_weekly_log_returns.fillna(0)

In [7]:
return_array = df_weekly_log_returns.to_numpy()
m_weekly_log = np.mean(return_array, axis=0)
S_weekly_log = np.cov(return_array.transpose())

In [8]:
m_log = 52 * m_weekly_log
S_log = 52 * S_weekly_log